# Feature-Based Multi-Layer Perceptron
Reference Paper: *Müller Jens et al. (2021) Coherent false seizure prediction in epilepsy, coincidence or providence?*

In [2]:
import sys
sys.path.append('..')

In [3]:
import pandas as pd
from models.load_data import load_features_and_labels
from utils.io import pickle_path
import os

# make it only use GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# If I don't do this, there are warnings
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from feature_extraction.extract_features import Features
from config.paths import PATHS

2025-12-19 19:30:27.572120: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
ptnt_dir = PATHS.patient_dirs()[0]
segs = pd.read_pickle(pickle_path(ptnt_dir.segments_table))
split = pd.read_pickle(pickle_path(ptnt_dir.train_test_split))

In [5]:
x_train, y_train, x_test, y_test = load_features_and_labels(segs, split, Features.ORDERED_FEATURE_NAMES)

In [6]:
n_features = len(Features.ORDERED_FEATURE_NAMES)
n_features

15

In [4]:
mlp_model = tf.keras.models.Sequential([
    Input([n_features]),
    Dense(16, activation='relu', name='dense0'),
    BatchNormalization(name='batch_norm0'),
    Dense(8, activation='relu', name='dense1'),
    BatchNormalization(name='batch_norm1'),
    Dense(4, activation='relu', name='dense2'),
    BatchNormalization(name='batch_norm2'),
    Dense(1, activation='sigmoid', name='output')
], name='MLP_model')

mlp_model.summary()

I0000 00:00:1766158073.396104 3995092 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22377 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:31:00.0, compute capability: 8.6


Model: "MLP_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense0 (Dense)                  │ (None, 16)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm0                     │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm1                     │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_norm2                     │ (None, 4)              │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 545 (2.13 KB)

 Trainable params: 489 (1.91 KB)

 Non-trainable params: 56 (224.00 B)

In [ ]:
mlp_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),  # todo sample weight
                  metrics=['accuracy'])

In [ ]:
mlp_model.fit(x_train, y_train, epochs=500)

In [ ]:
mlp_model.evaluate(x_test, y_test)

In [ ]:
# todo mean of an ensemble of 100 networks with different initial weights